In [1]:
import threading
import serial
import time
import matplotlib.pyplot as plt 
import numpy as np
import pyautogui
from bitstring import BitArray
from collections import deque
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPool1D
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestRegressor


import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# loading neural net
nn_pos = load_model('pickled_models/pos_nn/')
nn_pos._make_predict_function()
global graph
graph = tf.get_default_graph()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
# leading adaboost model
with open('pickled_models/adaboost_x.pkl', 'rb') as f:
    
    ab_x = pickle.load(f,)

with open('pickled_models/adaboost_y.pkl', 'rb') as f:
    
    ab_y = pickle.load(f)

In [5]:
# loading random forest model
with open('pickled_models/rf_x.pkl', 'rb') as f:
    
    rf_x = pickle.load(f,)

with open('pickled_models/rf_y.pkl', 'rb') as f:
    
    rf_y = pickle.load(f)

In [6]:
# loading linear regression model
with open('pickled_models/linear_x.pkl', 'rb') as f:
    
    lr_x = pickle.load(f,)

with open('pickled_models/linear_y.pkl', 'rb') as f:
    
    lr_y = pickle.load(f)

In [7]:
# loading decision tree model
with open('pickled_models/dt_x.pkl', 'rb') as f:
    
    dt_x = pickle.load(f,)

with open('pickled_models/dt_y.pkl', 'rb') as f:
    
    dt_y = pickle.load(f)

In [8]:
# setting up recording parameters and deques to record into

num_channel = 4

gain = 1

muscle_data = deque(maxlen = 500*num_channel + 994)

simple_muscle_data = deque(maxlen = 500*num_channel)

to_predict = deque()

current = deque()


In [10]:
# a function that recording muscle data mouse position and stores it in a complex deque
def measure_muscle():
    ser = serial.Serial('COM4', 230400)
    global muscle_data
    global to_predict
#     global current
    while run:
        first = ser.read(1)
        if int(BitArray(first).bin[0]):
            temp = deque()
            temp.append(int((BitArray(first).bin[1:] + BitArray(ser.read(1)).bin), 2))
            temp.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            temp.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            temp.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            cur_pos = np.array(pyautogui.position())
            temp.append(cur_pos[0])
            temp.append(cur_pos[1])
            temp.reverse()
            muscle_data += temp
            
            current = deque()
            current.append(int((BitArray(first).bin[1:] + BitArray(ser.read(1)).bin), 2))
            current.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            current.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            current.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            
            muscle_data.reverse()
            to_predict = current + muscle_data
            


            
# a simple function that stores onlt muscle data in a simple deque
def simple_measure_muscle():
    ser = serial.Serial('COM4', 230400)
    while run:
        first = ser.read(1)
        if int(BitArray(first).bin[0]):
            simple_muscle_data.append(int((BitArray(first).bin[1:] + BitArray(ser.read(1)).bin), 2))
            simple_muscle_data.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            simple_muscle_data.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))
            simple_muscle_data.append(int((BitArray(ser.read(1)).bin + BitArray(ser.read(1)).bin), 2))



# simpler model functions            
def move_mouse_ab():
    while run:
        pyautogui.moveRel(xOffset=(ab_x.predict([simple_muscle_data]) * gain), yOffset=(ab_y.predict([simple_muscle_data]) * gain))
        
        
def move_mouse_lr():
    while run:
        pyautogui.moveRel(xOffset=(lr_x.predict([muscle_data]) * gain), yOffset=(lr_y.predict([muscle_data]) * gain))

def move_mouse_rf():
    while run:
#         print(rf_x.predict([simple_muscle_data]), rf_y.predict([simple_muscle_data]))
    
        pyautogui.moveRel(xOffset=(rf_x.predict([simple_muscle_data]) * gain), yOffset=(rf_y.predict([simple_muscle_data]) * gain))

def move_mouse_dt():
    while run:
#         print(dt_x.predict([simple_muscle_data]), dt_y.predict([simple_muscle_data]))
    
        pyautogui.moveRel(xOffset=(dt_x.predict([simple_muscle_data]) * gain), yOffset=(dt_y.predict([simple_muscle_data]) * gain))

        
        
        
# complex model functions        
def move_mouse_nn_pos():
    with graph.as_default():
        while run:
#             muscle_data.reverse()
#             pyautogui.move([int(val) for val in nn_pos.predict(np.array(to_predict)[None, :])[0]])
            print([int(val) for val in nn_pos.predict(np.array(to_predict)[None, :])[0]])
#             muscle_data.reverse()

In [ ]:
# the final controll loops

run = True
lock = threading.Lock()

t1 = threading.Thread(target=simple_measure_muscle,)
t2 = threading.Thread(target=move_mouse_rf)

t1.start()

# wait for the deques to be full of data
time.sleep(5)

t2.start()

time.sleep(20)

run = False

t1.join()
t2.join()